In [1]:
%autosave 10
import time

Autosaving every 10 seconds


In [2]:
import openpyxl
import pandas as pd

# Please ensure that the Length of the Final Cuts are rounded off considerably

In case of mismatch due to algorithm rounding off after the Pattern Generation sheet is obtained, it may be manually matched

In [3]:
DashBoard_Obj=pd.read_excel("DashBoard.xlsx","Objective",index_col=0)
print(DashBoard_Obj)

                                                    Value  \
Parameter                                                   
Problem                                                20   
Objective                                               4   
Number of Combinations to be Generated                 50   
Timelimit for each of the smallest problems            30   
Timelimit for the Large CSP problems                  500   
Maximum Number of Unique Standard Length usages         2   
Maximum Number of Unique Pattern usages per Sta...     50   

                                                                                              Remarks  
Parameter                                                                                              
Problem                                                                  Diameter of the Bars (in mm)  
Objective                                           Case Type 4 = Generating the Top Minimum Waste...  
Number of Combinations to be Gener

In [4]:
# Create a dictionary from the index and a specific column
Decisions = DashBoard_Obj['Value'].to_dict()
print(Decisions)
# Decisions["Maximum Number of (Unique) Standard Length usages"]

{'Problem': 20, 'Objective': 4, 'Number of Combinations to be Generated': 50, 'Timelimit for each of the smallest problems': 30, 'Timelimit for the Large CSP problems': 500, 'Maximum Number of Unique Standard Length usages': 2, 'Maximum Number of Unique Pattern usages per Standard Length': 50}


In [5]:
DashBoard_Dataset_Cuts=pd.read_excel("DashBoard.xlsx","Dataset_Cuts",index_col=0)
print(DashBoard_Dataset_Cuts)

                         Total Requirements
Lengths of Cuts Allowed                    
1.810                                     8
2.730                                    59
2.930                                  1443
2.950                                   954
3.030                                   332
3.130                                   612
3.893                                    24
3.911                                    36
3.951                                    48
3.953                                    48
4.046                                    24
4.230                                   332
4.332                                     8
4.337                                    32
4.345                                    24
4.364                                    29
4.503                                    24
4.549                                    24
5.755                                    14
6.455                                    32
6.492                           

In [6]:
DashBoard_Dataset_StandardLengths=pd.read_excel("DashBoard.xlsx","Dataset_StandardLengths",index_col=0)
#print(type(DashBoard_Dataset_StandardLengths))
print(DashBoard_Dataset_StandardLengths)

                                Associated Cost
Total Standard Lengths Allowed                 
7.5                                    0.235619
9.0                                    0.282743
10.5                                   0.329867
12.0                                   0.376991


In [7]:
length_of_cuts_allowed = DashBoard_Dataset_Cuts.index.tolist()
print(length_of_cuts_allowed)

[1.81, 2.73, 2.93, 2.95, 3.03, 3.13, 3.893, 3.911, 3.951, 3.953, 4.046, 4.23, 4.332, 4.337, 4.345, 4.364, 4.503, 4.549, 5.755, 6.455, 6.492, 6.497, 6.573, 6.586, 7.09, 7.326, 7.337, 7.35, 7.39, 7.391, 7.676, 7.766]


In [8]:
requirements = DashBoard_Dataset_Cuts["Total Requirements"]
print(requirements)
#requirements[1.2]

Lengths of Cuts Allowed
1.810       8
2.730      59
2.930    1443
2.950     954
3.030     332
3.130     612
3.893      24
3.911      36
3.951      48
3.953      48
4.046      24
4.230     332
4.332       8
4.337      32
4.345      24
4.364      29
4.503      24
4.549      24
5.755      14
6.455      32
6.492      16
6.497      32
6.573      32
6.586      32
7.090       4
7.326       8
7.337      32
7.350      16
7.390      16
7.391      10
7.676      24
7.766      24
Name: Total Requirements, dtype: int64


In [9]:
total_max_standardlengths_allowed = DashBoard_Dataset_StandardLengths.index.tolist()
print(total_max_standardlengths_allowed)

[7.5, 9.0, 10.5, 12.0]


In [10]:
combinations = range(1,int(Decisions["Number of Combinations to be Generated"])+1)
print(combinations)

range(1, 51)


In [11]:
#max_time_in_seconds = Decisions["Timelimit for each of the smallest problems"]

#Timelimit_for_each_final_CSP_problem_minimizing_WCSP = Decisions["Timelimit for the Large CSP problems"]

# Generating Combinations Depending on Objectives

In [12]:
import gurobipy as gp
from gurobipy import GRB

In [13]:
start_time_is_this=time.time()
print(start_time_is_this)

1702800081.4397547


# Constraints dependant on Objectives

In [14]:
if Decisions["Objective"] == 4:
    M = 10000000
    X_Standards = {}
    
    for max_length in total_max_standardlengths_allowed:
        print("Max Standard Length Allowed: ", max_length)
        X_store = {}
        
        for c in combinations:
            # Create a new Gurobi model
            prob = gp.Model("Stock Cutting with multiple Master Lengths for Minimum Waste")

            # Variables
            x = {}
            m = {}
            g = {}

            for l in length_of_cuts_allowed:
                x[l] = prob.addVar(lb=0, vtype=GRB.INTEGER, name=f'x[{l}]')

            # Objective: Minimize Waste
            prob.setObjective(max_length - gp.quicksum(x[l] * l for l in length_of_cuts_allowed), sense=GRB.MINIMIZE)

            # Constraint 1
            prob.addConstr(gp.quicksum(x[l] * l for l in length_of_cuts_allowed) <= max_length)

            # Constraint 2
            for cc in combinations:
                if cc < c:
                    for l in length_of_cuts_allowed:
                        m[cc, l] = prob.addVar(lb=0, name=f'm[{cc},{l}]')
                        g[cc, l] = prob.addVar(vtype=GRB.BINARY, name=f'g[{cc},{l}]')

                        prob.addConstr(m[cc, l] >= X_store[cc, l] - x[l])
                        prob.addConstr(m[cc, l] >= x[l] - X_store[cc, l])

                        prob.addConstr(x[l] - X_store[cc, l] <= M * (1 - g[cc, l]))
                        prob.addConstr(X_store[cc, l] - x[l] <= M * g[cc, l])

                        prob.addConstr(m[cc, l] <= X_store[cc, l] - x[l] + M * (1 - g[cc, l]))
                        prob.addConstr(m[cc, l] <= x[l] - X_store[cc, l] + M * g[cc, l])

                    prob.addConstr(gp.quicksum(m[cc, l] for l in length_of_cuts_allowed) >= 1)

            # Solve the problem
            #prob.Params.TimeLimit = max_time_in_seconds
            prob.optimize()
            waste_generated = prob.objVal

            for l in length_of_cuts_allowed:
                X_store[c, l] = x[l].x
                if X_store[c, l] > 0:
                    print("\t X_store[", c, ",", l, "] = ", X_store[c, l])

            print("\t \t Total Waste Generated: ", waste_generated)
            X_store[c, "Waste Objective"] = waste_generated
            print("\t \t Total Time Taken: ", round(prob.Runtime, 1), " seconds, for this smallest problem of Standard length: ", max_length)

        X_Standards[max_length] = X_store.copy()

Max Standard Length Allowed:  7.5
Set parameter Username
Academic license - for non-commercial use only - expires 2024-12-10
Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (win64)

CPU model: Intel(R) Core(TM) i3-10110U CPU @ 2.10GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 1 rows, 32 columns and 32 nonzeros
Model fingerprint: 0x0032b6b5
Variable types: 0 continuous, 32 integer (0 binary)
Coefficient statistics:
  Matrix range     [2e+00, 8e+00]
  Objective range  [2e+00, 8e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [8e+00, 8e+00]
Found heuristic solution: objective 0.2600000
Presolve removed 0 rows and 13 columns
Presolve time: 0.00s
Presolved: 1 rows, 19 columns, 19 nonzeros
Variable types: 0 continuous, 19 integer (13 binary)
Found heuristic solution: objective 0.1090000

Root relaxation: objective 0.000000e+00, 1 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    C

     0     0    0.00000    0    4    0.03000    0.00000   100%     -    0s
     0     0    0.00000    0    3    0.03000    0.00000   100%     -    0s
     0     0    0.00000    0    3    0.03000    0.00000   100%     -    0s
     0     2    0.00000    0    3    0.03000    0.00000   100%     -    0s
*   71     2              11       0.0250000    0.00000   100%   2.7    0s
*   89     0              17       0.0210000    0.02100  0.00%   2.8    0s

Cutting planes:
  Gomory: 1
  Cover: 2
  MIR: 2
  StrongCG: 3

Explored 90 nodes (287 simplex iterations) in 0.09 seconds (0.01 work units)
Thread count was 4 (of 4 available processors)

Solution count 3: 0.021 0.025 0.03 

Optimal solution found (tolerance 1.00e-04)
Best objective 2.100000000000e-02, best bound 2.100000000000e-02, gap 0.0000%
	 X_store[ 4 , 2.93 ] =  1.0
	 X_store[ 4 , 4.549 ] =  1.0
	 	 Total Waste Generated:  0.02099999999999902
	 	 Total Time Taken:  0.1  seconds, for this smallest problem of Standard length:  7.5
Gurobi 

Best objective 3.300000000000e-02, best bound 3.300000000000e-02, gap 0.0000%
	 X_store[ 7 , 3.13 ] =  1.0
	 X_store[ 7 , 4.337 ] =  1.0
	 	 Total Waste Generated:  0.03300000000000036
	 	 Total Time Taken:  0.1  seconds, for this smallest problem of Standard length:  7.5
Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (win64)

CPU model: Intel(R) Core(TM) i3-10110U CPU @ 2.10GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 1352 rows, 480 columns and 3392 nonzeros
Model fingerprint: 0xfb5b3fba
Variable types: 224 continuous, 256 integer (224 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+07]
  Objective range  [2e+00, 8e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+07]
Presolve removed 1278 rows and 439 columns
Presolve time: 0.00s
Presolved: 74 rows, 41 columns, 284 nonzeros
Variable types: 0 continuous, 41 integer (33 binary)
Found heuristic solution: objectiv

Found heuristic solution: objective 1.2400000
Found heuristic solution: objective 1.1410000
Found heuristic solution: objective 0.2060000

Root relaxation: objective 0.000000e+00, 25 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    0.00000    0    1    0.20600    0.00000   100%     -    0s
H    0     0                       0.1060000    0.00000   100%     -    0s
     0     0    0.00000    0    3    0.10600    0.00000   100%     -    0s
     0     0    0.00000    0    2    0.10600    0.00000   100%     -    0s
     0     0    0.00000    0    1    0.10600    0.00000   100%     -    0s
     0     0    0.00000    0    4    0.10600    0.00000   100%     -    0s
     0     0    0.00000    0    4    0.10600    0.00000   100%     -    0s
     0     0    0.00000    0    5    0.10600    0.00000   100%     -    0s
     0     0    0.0000

  StrongCG: 5
  Relax-and-lift: 1

Explored 118 nodes (349 simplex iterations) in 0.17 seconds (0.04 work units)
Thread count was 4 (of 4 available processors)

Solution count 2: 0.125 0.75 

Optimal solution found (tolerance 1.00e-04)
Best objective 1.250000000000e-01, best bound 1.250000000000e-01, gap 0.0000%
	 X_store[ 14 , 3.03 ] =  1.0
	 X_store[ 14 , 4.345 ] =  1.0
	 	 Total Waste Generated:  0.125
	 	 Total Time Taken:  0.2  seconds, for this smallest problem of Standard length:  7.5
Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (win64)

CPU model: Intel(R) Core(TM) i3-10110U CPU @ 2.10GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 2703 rows, 928 columns and 6752 nonzeros
Model fingerprint: 0xe10029d4
Variable types: 448 continuous, 480 integer (448 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+07]
  Objective range  [2e+00, 8e+00]
  Bounds range     [1e+00, 1e+00]
  RHS 

...


Optimal solution found (tolerance 1.00e-04)
Best objective 6.600000000000e-02, best bound 6.600000000000e-02, gap 0.0000%
	 X_store[ 36 , 1.81 ] =  1.0
	 X_store[ 36 , 2.73 ] =  1.0
	 X_store[ 36 , 3.03 ] =  1.0
	 X_store[ 36 , 4.364 ] =  1.0
	 	 Total Waste Generated:  0.06599999999999895
	 	 Total Time Taken:  2.0  seconds, for this smallest problem of Standard length:  12.0
Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (win64)

CPU model: Intel(R) Core(TM) i3-10110U CPU @ 2.10GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 6949 rows, 2336 columns and 17312 nonzeros
Model fingerprint: 0x94f491ae
Variable types: 1152 continuous, 1184 integer (1152 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+07]
  Objective range  [2e+00, 8e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+07]
Presolve removed 6366 rows and 2122 columns
Presolve time: 0.09s
Presolved: 583 row

  RHS range        [1e+00, 1e+07]
Presolve removed 6906 rows and 2302 columns
Presolve time: 0.14s
Presolved: 622 rows, 226 columns, 2832 nonzeros
Variable types: 0 continuous, 226 integer (138 binary)
Found heuristic solution: objective 0.8100000

Root relaxation: objective 0.000000e+00, 195 iterations, 0.01 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    0.00000    0    6    0.81000    0.00000   100%     -    0s
H    0     0                       0.0710000    0.00000   100%     -    0s
     0     0    0.00000    0   11    0.07100    0.00000   100%     -    0s
     0     0    0.00000    0   17    0.07100    0.00000   100%     -    0s
     0     0    0.00000    0   17    0.07100    0.00000   100%     -    0s
     0     2    0.00000    0   17    0.07100    0.00000   100%     -    0s

Cutting planes:
  Learned: 19
  Gomory: 2
  Cover: 8
  MIR: 

Thread count was 4 (of 4 available processors)

Solution count 5: 0.077 0.078 0.106 ... 0.801

Optimal solution found (tolerance 1.00e-04)
Best objective 7.700000000000e-02, best bound 7.700000000000e-02, gap 0.0000%
	 X_store[ 43 , 1.81 ] =  1.0
	 X_store[ 43 , 3.03 ] =  1.0
	 X_store[ 43 , 3.13 ] =  1.0
	 X_store[ 43 , 3.953 ] =  1.0
	 	 Total Waste Generated:  0.07700000000000173
	 	 Total Time Taken:  1.8  seconds, for this smallest problem of Standard length:  12.0
Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (win64)

CPU model: Intel(R) Core(TM) i3-10110U CPU @ 2.10GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 8300 rows, 2784 columns and 20672 nonzeros
Model fingerprint: 0xd58d9442
Variable types: 1376 continuous, 1408 integer (1376 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+07]
  Objective range  [2e+00, 8e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+

Found heuristic solution: objective 0.1800000

Root relaxation: objective 1.776357e-15, 250 iterations, 0.01 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    0.00000    0    8    0.18000    0.00000   100%     -    0s
     0     0    0.00000    0   10    0.18000    0.00000   100%     -    0s
     0     0    0.00000    0    6    0.18000    0.00000   100%     -    0s
     0     0    0.00000    0    6    0.18000    0.00000   100%     -    0s
H    0     0                       0.0840000    0.00000   100%     -    0s
     0     2    0.00000    0    6    0.08400    0.00000   100%     -    0s
*  578    66              36       0.0800000    0.00000   100%   8.9    0s

Cutting planes:
  Learned: 59
  Gomory: 2
  Cover: 2
  Implied bound: 7

Explored 2191 nodes (24859 simplex iterations) in 1.82 seconds (0.54 work units)
Thread count was 4 (of 4 availabl

  Learned: 115
  Gomory: 1
  Cover: 9
  Implied bound: 8

Explored 2465 nodes (28475 simplex iterations) in 2.27 seconds (0.66 work units)
Thread count was 4 (of 4 available processors)

Solution count 5: 0.08 0.082 0.085 ... 0.16

Optimal solution found (tolerance 1.00e-04)
Best objective 8.000000000000e-02, best bound 8.000000000000e-02, gap 0.0000%
	 X_store[ 50 , 2.73 ] =  1.0
	 X_store[ 50 , 3.03 ] =  2.0
	 X_store[ 50 , 3.13 ] =  1.0
	 	 Total Waste Generated:  0.08000000000000185
	 	 Total Time Taken:  2.3  seconds, for this smallest problem of Standard length:  12.0


In [15]:
max_combination_for_each_standard = {}

if Decisions["Objective"] == 4:
    for max_length in total_max_standardlengths_allowed:
        print("Max Standard Length Allowed: ", max_length)
        X_store = {}
        combination_previous_end_number = Decisions["Number of Combinations to be Generated"]
        current_combination_number = combination_previous_end_number

        for ll in length_of_cuts_allowed:
            if ll > max_length:
                continue

            current_combination_number += 1

            # Create a new Gurobi model
            prob = gp.Model("Stock Cutting with multiple Master Lengths for Minimum Waste")

            # Variables
            x = {}
            m = {}
            g = {}

            for l in length_of_cuts_allowed:
                x[l] = prob.addVar(lb=0, vtype=GRB.INTEGER, name=f'x[{l}]')

            # Objective: Minimize Waste
            prob.setObjective(max_length - gp.quicksum(x[l] * l for l in length_of_cuts_allowed), sense=GRB.MINIMIZE)

            # Constraint 0
            prob.addConstr(x[ll] >= 1)

            # Constraint 1
            prob.addConstr(gp.quicksum(x[l] * l for l in length_of_cuts_allowed) <= max_length)

            # Constraint 2
            for cc in combinations:
                for l in length_of_cuts_allowed:
                    m[cc, l] = prob.addVar(lb=0, name=f'm[{cc},{l}]')
                    g[cc, l] = prob.addVar(vtype=GRB.BINARY, name=f'g[{cc},{l}]')

                    prob.addConstr(m[cc, l] >= X_Standards[max_length][cc, l] - x[l])
                    prob.addConstr(m[cc, l] >= x[l] - X_Standards[max_length][cc, l])

                    prob.addConstr(x[l] - X_Standards[max_length][cc, l] <= M * (1 - g[cc, l]))
                    prob.addConstr(X_Standards[max_length][cc, l] - x[l] <= M * g[cc, l])

                    prob.addConstr(m[cc, l] <= X_Standards[max_length][cc, l] - x[l] + M * (1 - g[cc, l]))
                    prob.addConstr(m[cc, l] <= x[l] - X_Standards[max_length][cc, l] + M * g[cc, l])

                prob.addConstr(gp.quicksum(m[cc, l] for l in length_of_cuts_allowed) >= 1)

            for cc in range(combination_previous_end_number + 1, current_combination_number):
                for l in length_of_cuts_allowed:
                    m[cc, l] = prob.addVar(lb=0, name=f'm[{cc},{l}]')
                    g[cc, l] = prob.addVar(vtype=GRB.BINARY, name=f'g[{cc},{l}]')

                    prob.addConstr(m[cc, l] >= X_store[cc, l] - x[l])
                    prob.addConstr(m[cc, l] >= x[l] - X_store[cc, l])

                    prob.addConstr(x[l] - X_store[cc, l] <= M * (1 - g[cc, l]))
                    prob.addConstr(X_store[cc, l] - x[l] <= M * g[cc, l])

                    prob.addConstr(m[cc, l] <= X_store[cc, l] - x[l] + M * (1 - g[cc, l]))
                    prob.addConstr(m[cc, l] <= x[l] - X_store[cc, l] + M * g[cc, l])

                prob.addConstr(gp.quicksum(m[cc, l] for l in length_of_cuts_allowed) >= 1)

            # Solve the problem
            #prob.Params.TimeLimit = max_time_in_seconds
            prob.optimize()
            
            
            
            if prob.SolCount:
                waste_generated = prob.objVal

                for l in length_of_cuts_allowed:
                    X_store[current_combination_number, l] = x[l].x
                    if X_store[current_combination_number, l] > 0:
                        print("\t X_store[", current_combination_number, ",", l, "] = ", X_store[current_combination_number, l])

                print("\t \t Total Waste Generated: ", waste_generated,
                      " during mandatory unique usage of cut-length: ", ll, "in standard length: ", max_length)
                X_store[current_combination_number, "Waste Objective"] = waste_generated
                print("\t \t Total Time Taken: ", round(prob.Runtime, 1), " seconds, for this smallest problem")
            else:
                current_combination_number-=1

        max_combination_for_each_standard[max_length] = current_combination_number
        X_Standards[max_length].update(X_store)

Max Standard Length Allowed:  7.5
Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (win64)

CPU model: Intel(R) Core(TM) i3-10110U CPU @ 2.10GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 9652 rows, 3232 columns and 24033 nonzeros
Model fingerprint: 0xfe6b8cf2
Variable types: 1600 continuous, 1632 integer (1600 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+07]
  Objective range  [2e+00, 8e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+07]
Presolve removed 9629 rows and 3220 columns
Presolve time: 0.04s
Presolved: 23 rows, 12 columns, 72 nonzeros
Variable types: 0 continuous, 12 integer (10 binary)
Found heuristic solution: objective 1.1500000
Found heuristic solution: objective 1.1410000

Root relaxation: objective 0.000000e+00, 8 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl | 


Optimal solution found (tolerance 1.00e-04)
Best objective 5.590000000000e-01, best bound 5.590000000000e-01, gap 0.0000%
	 X_store[ 55 , 3.03 ] =  1.0
	 X_store[ 55 , 3.911 ] =  1.0
	 	 Total Waste Generated:  0.5590000000000002  during mandatory unique usage of cut-length:  3.03 in standard length:  7.5
	 	 Total Time Taken:  0.1  seconds, for this smallest problem
Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (win64)

CPU model: Intel(R) Core(TM) i3-10110U CPU @ 2.10GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 10617 rows, 3552 columns and 26433 nonzeros
Model fingerprint: 0xa40c20e6
Variable types: 1760 continuous, 1792 integer (1760 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+07]
  Objective range  [2e+00, 8e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+07]
Presolve removed 10617 rows and 3552 columns
Presolve time: 0.04s
Presolve: All rows and col

	 X_store[ 62 , 1.81 ] =  1.0
	 X_store[ 62 , 4.23 ] =  1.0
	 	 Total Waste Generated:  1.459999999999999  during mandatory unique usage of cut-length:  4.23 in standard length:  7.5
	 	 Total Time Taken:  0.1  seconds, for this smallest problem
Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (win64)

CPU model: Intel(R) Core(TM) i3-10110U CPU @ 2.10GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 11968 rows, 4000 columns and 29793 nonzeros
Model fingerprint: 0xd334adb3
Variable types: 1984 continuous, 2016 integer (1984 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+07]
  Objective range  [2e+00, 8e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+07]
Presolve removed 11968 rows and 4000 columns
Presolve time: 0.03s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.07 seconds (0.02 work units)
Thread count was 1 (of 4 available pr

Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (win64)

CPU model: Intel(R) Core(TM) i3-10110U CPU @ 2.10GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 13319 rows, 4448 columns and 33153 nonzeros
Model fingerprint: 0xef19aa07
Variable types: 2208 continuous, 2240 integer (2208 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+07]
  Objective range  [2e+00, 8e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+07]
Presolve removed 13319 rows and 4448 columns
Presolve time: 0.02s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.06 seconds (0.01 work units)
Thread count was 1 (of 4 available processors)

Solution count 1: 1.045 

Optimal solution found (tolerance 1.00e-04)
Best objective 1.045000000000e+00, best bound 1.045000000000e+00, gap 0.0000%
	 X_store[ 70 , 6.455 ] =  1.0
	 	 Total Waste Generated:  1.045  during mandatory uniq

Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 14284 rows, 4768 columns and 35553 nonzeros
Model fingerprint: 0x24a78c30
Variable types: 2368 continuous, 2400 integer (2368 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+07]
  Objective range  [2e+00, 8e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+07]
Presolve removed 14219 rows and 4768 columns
Presolve time: 0.01s

Explored 0 nodes (0 simplex iterations) in 0.04 seconds (0.01 work units)
Thread count was 1 (of 4 available processors)

Solution count 0

Model is infeasible or unbounded
Best objective -, best bound -, gap -
Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (win64)

CPU model: Intel(R) Core(TM) i3-10110U CPU @ 2.10GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 14284 rows, 4768 columns and 35553 nonzeros
Model fingerprint: 0xc0c4e6a8
Variable typ

...


CPU model: Intel(R) Core(TM) i3-10110U CPU @ 2.10GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 11775 rows, 3936 columns and 29313 nonzeros
Model fingerprint: 0x524389b6
Variable types: 1952 continuous, 1984 integer (1952 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+07]
  Objective range  [2e+00, 8e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+07]
Presolve removed 11681 rows and 3886 columns
Presolve time: 0.06s
Presolved: 94 rows, 50 columns, 321 nonzeros
Variable types: 0 continuous, 50 integer (37 binary)
Found heuristic solution: objective 0.2800000
Found heuristic solution: objective 0.2050000

Root relaxation: objective 0.000000e+00, 30 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    0.00000 

Found heuristic solution: objective 0.4150000
Found heuristic solution: objective 0.1610000
Found heuristic solution: objective 0.1240000

Root relaxation: objective 0.000000e+00, 43 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    0.00000    0    2    0.12400    0.00000   100%     -    0s
     0     0    0.00000    0    4    0.12400    0.00000   100%     -    0s
     0     0    0.00000    0    9    0.12400    0.00000   100%     -    0s
H    0     0                       0.0850000    0.00000   100%     -    0s
     0     0    0.00000    0    7    0.08500    0.00000   100%     -    0s
     0     0    0.00000    0    7    0.08500    0.00000   100%     -    0s
     0     0    0.00000    0    4    0.08500    0.00000   100%     -    0s
     0     0    0.00000    0    2    0.08500    0.00000   100%     -    0s
     0     0    0.0000


     0     0   -0.00000    0    1    0.17200   -0.00000   100%     -    0s
     0     0   -0.00000    0    2    0.17200   -0.00000   100%     -    0s
     0     0   -0.00000    0    4    0.17200   -0.00000   100%     -    0s
     0     0   -0.00000    0    4    0.17200   -0.00000   100%     -    0s
     0     0   -0.00000    0    4    0.17200   -0.00000   100%     -    0s
H    0     0                       0.1370000   -0.00000   100%     -    0s
     0     2   -0.00000    0    4    0.13700   -0.00000   100%     -    0s
*  145     0              17       0.1010000    0.10100  0.00%   2.0    0s

Cutting planes:
  Cover: 10
  MIR: 1
  StrongCG: 1
  Relax-and-lift: 1

Explored 146 nodes (339 simplex iterations) in 0.30 seconds (0.05 work units)
Thread count was 4 (of 4 available processors)

Solution count 4: 0.101 0.137 0.172 0.901 

Optimal solution found (tolerance 1.00e-04)
Best objective 1.010000000000e-01, best bound 1.010000000000e-01, gap 0.0000%
	 X_store[ 68 , 4.549 ] =  1.0
	 X


Solution count 2: 0.563 0.963 

Optimal solution found (tolerance 1.00e-04)
Best objective 5.630000000000e-01, best bound 5.630000000000e-01, gap 0.0000%
	 X_store[ 72 , 1.81 ] =  1.0
	 X_store[ 72 , 3.13 ] =  1.0
	 X_store[ 72 , 6.497 ] =  1.0
	 	 Total Waste Generated:  0.5630000000000006  during mandatory unique usage of cut-length:  6.497 in standard length:  12.0
	 	 Total Time Taken:  0.1  seconds, for this smallest problem
Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (win64)

CPU model: Intel(R) Core(TM) i3-10110U CPU @ 2.10GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 13898 rows, 4640 columns and 34593 nonzeros
Model fingerprint: 0x594da259
Variable types: 2304 continuous, 2336 integer (2304 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+07]
  Objective range  [2e+00, 8e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+07]
Presolve removed 13891 rows 

Best objective 1.140000000000e-01, best bound 1.140000000000e-01, gap 0.0000%
	 X_store[ 77 , 4.549 ] =  1.0
	 X_store[ 77 , 7.337 ] =  1.0
	 	 Total Waste Generated:  0.11400000000000077  during mandatory unique usage of cut-length:  7.337 in standard length:  12.0
	 	 Total Time Taken:  0.1  seconds, for this smallest problem
Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (win64)

CPU model: Intel(R) Core(TM) i3-10110U CPU @ 2.10GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 14863 rows, 4960 columns and 36993 nonzeros
Model fingerprint: 0xfd1e71ca
Variable types: 2464 continuous, 2496 integer (2464 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+07]
  Objective range  [2e+00, 8e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+07]
Presolve removed 14856 rows and 4955 columns
Presolve time: 0.04s
Presolved: 7 rows, 5 columns, 19 nonzeros
Variable types: 0 continu

In [16]:
import winsound
winsound.Beep(500,1000)
max_combination_for_each_standard

{7.5: 74, 9.0: 82, 10.5: 82, 12.0: 82}

In [17]:
wb=openpyxl.Workbook()

sheet_for_Combinations = wb.active

row_number=1
sheet_for_Combinations.cell(row=row_number,column=1).value= "Length of Cut"

column_number=2
for i,max_length in enumerate(total_max_standardlengths_allowed):
    #for c in range(1,current_combination_number+1):
    for c in range(1,max_combination_for_each_standard[max_length]+1):
        sheet_for_Combinations.cell(row=row_number,column= column_number).value="Combination "+str(column_number-1)
        column_number+=1




for l in length_of_cuts_allowed:
    row_number+=1
    sheet_for_Combinations.cell(row=row_number,column=1).value= l
    column_number=2
    for i,max_length in enumerate(total_max_standardlengths_allowed):
        #for c in range(1,current_combination_number+1):
        for c in range(1,max_combination_for_each_standard[max_length]+1):
            sheet_for_Combinations.cell(row=row_number,column= column_number).value= X_Standards[max_length][c,l]
            column_number+=1
        
        
row_number+=1
sheet_for_Combinations.cell(row=row_number,column=1).value= "Standard Max Length"
column_number=2
for i,max_length in enumerate(total_max_standardlengths_allowed):
    #for c in range(1,current_combination_number+1):
    for c in range(1,max_combination_for_each_standard[max_length]+1):
        sheet_for_Combinations.cell(row=row_number,column= column_number).value= max_length
        column_number+=1
        

        
row_number+=1
sheet_for_Combinations.cell(row=row_number,column=1).value= "Wasted Length"
column_number=2
for i,max_length in enumerate(total_max_standardlengths_allowed):
    #for c in range(1,current_combination_number+1):
    for c in range(1,max_combination_for_each_standard[max_length]+1):
        sheet_for_Combinations.cell(row=row_number,column= column_number).value= X_Standards[max_length][c,"Waste Objective"]
        column_number+=1

In [18]:
#file_path = "./Problem_"+str(Decisions["Problem"])+"__CombinationGeneration_"+str(Decisions["Number of Combinations to be Generated"])+"__TimeLimit_"+str(max_time_in_seconds)+".xlsx"
file_path = "./Problem_"+str(Decisions["Problem"])+"__CombinationGeneration_"+str(Decisions["Number of Combinations to be Generated"])+"__TimeLimit_"+".xlsx"
wb.save(file_path)
print(file_path)

./Problem_20__CombinationGeneration_50__TimeLimit_.xlsx


In [19]:
X_Standards

{7.5: {(1, 1.81): 0.0,
  (1, 2.73): 0.0,
  (1, 2.93): 0.0,
  (1, 2.95): 1.0,
  (1, 3.03): 0.0,
  (1, 3.13): 0.0,
  (1, 3.893): 0.0,
  (1, 3.911): 0.0,
  (1, 3.951): 0.0,
  (1, 3.953): 0.0,
  (1, 4.046): 0.0,
  (1, 4.23): 0.0,
  (1, 4.332): 0.0,
  (1, 4.337): 0.0,
  (1, 4.345): 0.0,
  (1, 4.364): 0.0,
  (1, 4.503): 0.0,
  (1, 4.549): 1.0,
  (1, 5.755): -0.0,
  (1, 6.455): -0.0,
  (1, 6.492): -0.0,
  (1, 6.497): -0.0,
  (1, 6.573): -0.0,
  (1, 6.586): -0.0,
  (1, 7.09): -0.0,
  (1, 7.326): -0.0,
  (1, 7.337): -0.0,
  (1, 7.35): -0.0,
  (1, 7.39): -0.0,
  (1, 7.391): -0.0,
  (1, 7.676): 0.0,
  (1, 7.766): 0.0,
  (1, 'Waste Objective'): 0.0009999999999994458,
  (2, 1.81): -0.0,
  (2, 2.73): -0.0,
  (2, 2.93): -0.0,
  (2, 2.95): -0.0,
  (2, 3.03): -0.0,
  (2, 3.13): 1.0,
  (2, 3.893): 0.0,
  (2, 3.911): 0.0,
  (2, 3.951): -0.0,
  (2, 3.953): 0.0,
  (2, 4.046): 0.0,
  (2, 4.23): 0.0,
  (2, 4.332): 0.0,
  (2, 4.337): 0.0,
  (2, 4.345): 0.0,
  (2, 4.364): 1.0,
  (2, 4.503): -0.0,
  (2, 4.549):

In [20]:
for max_length in total_max_standardlengths_allowed:
    #print(max_length)
    print(X_Standards[max_length])

{(1, 1.81): 0.0, (1, 2.73): 0.0, (1, 2.93): 0.0, (1, 2.95): 1.0, (1, 3.03): 0.0, (1, 3.13): 0.0, (1, 3.893): 0.0, (1, 3.911): 0.0, (1, 3.951): 0.0, (1, 3.953): 0.0, (1, 4.046): 0.0, (1, 4.23): 0.0, (1, 4.332): 0.0, (1, 4.337): 0.0, (1, 4.345): 0.0, (1, 4.364): 0.0, (1, 4.503): 0.0, (1, 4.549): 1.0, (1, 5.755): -0.0, (1, 6.455): -0.0, (1, 6.492): -0.0, (1, 6.497): -0.0, (1, 6.573): -0.0, (1, 6.586): -0.0, (1, 7.09): -0.0, (1, 7.326): -0.0, (1, 7.337): -0.0, (1, 7.35): -0.0, (1, 7.39): -0.0, (1, 7.391): -0.0, (1, 7.676): 0.0, (1, 7.766): 0.0, (1, 'Waste Objective'): 0.0009999999999994458, (2, 1.81): -0.0, (2, 2.73): -0.0, (2, 2.93): -0.0, (2, 2.95): -0.0, (2, 3.03): -0.0, (2, 3.13): 1.0, (2, 3.893): 0.0, (2, 3.911): 0.0, (2, 3.951): -0.0, (2, 3.953): 0.0, (2, 4.046): 0.0, (2, 4.23): 0.0, (2, 4.332): 0.0, (2, 4.337): 0.0, (2, 4.345): 0.0, (2, 4.364): 1.0, (2, 4.503): -0.0, (2, 4.549): -0.0, (2, 5.755): -0.0, (2, 6.455): -0.0, (2, 6.492): -0.0, (2, 6.497): 0.0, (2, 6.573): 0.0, (2, 6.586):

# Main CSProblem starts here:~

(designed to also run separately from the above program, be cautious about the intermediate file name changes if done manually)

In [21]:
import gurobipy as gp
from gurobipy import GRB

from math import ceil

In [22]:
patterns_df = pd.read_excel(file_path, index_col=0)
#print(patterns_df)

In [23]:
# Get the last two rows into a new DataFrame
last_two_rows = patterns_df.tail(2).copy()  # tail(2) gets the last 2 rows

# Remove the last two rows from the original DataFrame
new_df = patterns_df.drop(patterns_df.tail(2).index)

In [24]:
# Find unique elements in the first row (index 0)
unique_elements_row = last_two_rows.iloc[0].unique()

print("Unique elements in the first row of last_two_rows:")
print(unique_elements_row)

Unique elements in the first row of last_two_rows:
[ 7.5  9.  10.5 12. ]


In [25]:
all_length_of_cuts_allowed = new_df.index.tolist() # we could have used length_of_cuts_allowed, but here we take data from the Excel Sheet as we may want to change something for Later (in the Excel Sheet manually) which is however not recommended nor necessary
all_length_of_cuts_allowed

[1.81,
 2.73,
 2.93,
 2.95,
 3.03,
 3.13,
 3.893,
 3.911,
 3.951,
 3.953,
 4.046,
 4.23,
 4.332,
 4.337,
 4.345,
 4.364,
 4.503,
 4.549,
 5.755,
 6.455,
 6.492,
 6.497,
 6.573,
 6.586,
 7.09,
 7.326,
 7.337,
 7.35,
 7.39,
 7.391,
 7.676,
 7.766]

# Waste, Cost, Standard, Pattern: The WCSP sequence of Minimization (4 sequential problems

In [26]:
# Constraints
sl_key__value_c_mapping_dict={}
set_of_encountered_sl=set()
for c in patterns_df:
    sl = last_two_rows[c]["Standard Max Length"]
    if sl in set_of_encountered_sl:
        sl_key__value_c_mapping_dict[sl].add(c)
    else:
        sl_key__value_c_mapping_dict[sl]={c}
        set_of_encountered_sl.add(sl)
    
#print(sl_key__value_c_mapping_dict)

# WASTE MINIMIZATION

In [27]:
# Create a Gurobi model
CSProb = gp.Model("Stock Cutting with Waste Minimization")

# Variables
x = {}

for c in patterns_df:
    x[c] = CSProb.addVar(lb=0, vtype=GRB.INTEGER, name=f'x[{c}]')


# Objective: Minimize Waste
CSProb.setObjective(gp.quicksum(x[c] * last_two_rows[c]["Wasted Length"] for c in patterns_df), sense=GRB.MINIMIZE)

# Satisfying Requirements
for l in all_length_of_cuts_allowed:
    CSProb.addConstr(gp.quicksum(x[c] * new_df[c][l] for c in patterns_df) >= requirements[l])





# Set a time limit for optimization (e.g., 60 seconds)
#https://www.gurobi.com/documentation/10.0/refman/python_parameter_examples.html#PythonParameterExamples
currentTIMElimit = CSProb.Params.timeLimit
print("Default Time Limit: ",currentTIMElimit)
#CSProb.Params.timeLimit = Timelimit_for_each_final_CSP_problem_minimizing_WCSP


currentIntFeasTolerancelimit = CSProb.Params.IntFeasTol
print("Default Interger Tolerance: ",currentIntFeasTolerancelimit)
CSProb.Params.IntFeasTol = 1e-9


#FeasibilityTol
#https://www.gurobi.com/documentation/current/refman/feasibilitytol.html#parameter:FeasibilityTol
currentFeasibilityTolerancelimit = CSProb.Params.FeasibilityTol
print("Default Feasibility Tolerance: ",currentFeasibilityTolerancelimit)
CSProb.Params.FeasibilityTol = 1e-9


#OptimalityTol
#https://www.gurobi.com/documentation/current/refman/optimalitytol.html
currentOptimalityTolerancelimit = CSProb.Params.OptimalityTol
print("Default Optimality Tolerance: ",currentOptimalityTolerancelimit)
CSProb.Params.OptimalityTol = 1e-9


currentMIPGapLimit = CSProb.Params.MIPGap
print("Default MIP Gap Limit: ",currentMIPGapLimit)
#CSProb.setParam('MIPGap', 0)
CSProb.Params.MIPGap = 0
print("Assigned MIP Gap Limit: ",currentMIPGapLimit)

#https://www.gurobi.com/documentation/current/refman/mip_models.html#:~:text=The%20MIPFocus%20parameter%20allows%20you%20to%20modify%20your,good%20quality%20feasible%20solutions%2C%20you%20can%20select%20MIPFocus%3D1.
#https://www.gurobi.com/documentation/current/refman/mipfocus.html
#https://support.gurobi.com/hc/en-us/community/posts/8791754564369-How-does-MIPFocus-really-work-
currentMIPFocus = CSProb.Params.MIPFocus
print("The current MIPFocus is: ",currentMIPFocus)
CSProb.Params.MIPFocus=2


# Optimize the model
CSProb.optimize()

Default Time Limit:  inf
Default Interger Tolerance:  1e-05
Set parameter IntFeasTol to value 1e-09
Default Feasibility Tolerance:  1e-06
Set parameter FeasibilityTol to value 1e-09
Default Optimality Tolerance:  1e-06
Set parameter OptimalityTol to value 1e-09
Default MIP Gap Limit:  0.0001
Set parameter MIPGap to value 0
Assigned MIP Gap Limit:  0.0001
The current MIPFocus is:  0
Set parameter MIPFocus to value 2
Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (win64)

CPU model: Intel(R) Core(TM) i3-10110U CPU @ 2.10GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 32 rows, 320 columns and 751 nonzeros
Model fingerprint: 0xa06ba730
Variable types: 0 continuous, 320 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 5e+00]
  Objective range  [1e-03, 2e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [4e+00, 1e+03]
Found heuristic solution: objective 334.2050000
Presolve 

# COST MINIMIZATION

In [28]:
# Create a new Gurobi model
CSProb2 = gp.Model("Stock Cutting with Cost Minimization")

# Variables
x_2 = {}

for c in patterns_df:
    x_2[c] = CSProb2.addVar(lb=0, vtype=GRB.INTEGER, name=f'x_2[{c}]')



# Previous OBJECTIVE 1: Minimize Waste : Now treated as constraint
CSProb2.addConstr(gp.quicksum(x_2[c]*last_two_rows[c]["Wasted Length"] for c in patterns_df) <= ceil(CSProb.objVal *1000)/1000, name="Obj1 as Const.")


for l in all_length_of_cuts_allowed:
    CSProb2.addConstr(gp.quicksum(x_2[c] * new_df[c][l] for c in patterns_df) >= requirements[l])


# Objective 2
CSProb2.setObjective(gp.quicksum(x_2[c] * DashBoard_Dataset_StandardLengths["Associated Cost"][last_two_rows[c]["Standard Max Length"]] for c in patterns_df),
                     sense=GRB.MINIMIZE)






# Set a time limit for optimization (e.g., 60 seconds)
#https://www.gurobi.com/documentation/10.0/refman/python_parameter_examples.html#PythonParameterExamples
currentTIMElimit = CSProb2.Params.timeLimit
print("Default Time Limit: ",currentTIMElimit)
#CSProb2.Params.timeLimit = Timelimit_for_each_final_CSP_problem_minimizing_WCSP


currentIntFeasTolerancelimit = CSProb2.Params.IntFeasTol
print("Default Interger Tolerance: ",currentIntFeasTolerancelimit)
CSProb2.Params.IntFeasTol = 1e-9


#FeasibilityTol
#https://www.gurobi.com/documentation/current/refman/feasibilitytol.html#parameter:FeasibilityTol
currentFeasibilityTolerancelimit = CSProb2.Params.FeasibilityTol
print("Default Feasibility Tolerance: ",currentFeasibilityTolerancelimit)
CSProb2.Params.FeasibilityTol = 1e-9


#OptimalityTol
#https://www.gurobi.com/documentation/current/refman/optimalitytol.html
currentOptimalityTolerancelimit = CSProb2.Params.OptimalityTol
print("Default Optimality Tolerance: ",currentOptimalityTolerancelimit)
CSProb2.Params.OptimalityTol = 1e-9


currentMIPGapLimit = CSProb2.Params.MIPGap
print("Default MIP Gap Limit: ",currentMIPGapLimit)
#CSProb.setParam('MIPGap', 0)
CSProb2.Params.MIPGap = 0
print("Assigned MIP Gap Limit: ",currentMIPGapLimit)

#https://www.gurobi.com/documentation/current/refman/mip_models.html#:~:text=The%20MIPFocus%20parameter%20allows%20you%20to%20modify%20your,good%20quality%20feasible%20solutions%2C%20you%20can%20select%20MIPFocus%3D1.
#https://www.gurobi.com/documentation/current/refman/mipfocus.html
#https://support.gurobi.com/hc/en-us/community/posts/8791754564369-How-does-MIPFocus-really-work-
currentMIPFocus = CSProb2.Params.MIPFocus
print("The current MIPFocus is: ",currentMIPFocus)
CSProb2.Params.MIPFocus=2


# Optimize the model
CSProb2.optimize()

Default Time Limit:  inf
Default Interger Tolerance:  1e-05
Set parameter IntFeasTol to value 1e-09
Default Feasibility Tolerance:  1e-06
Set parameter FeasibilityTol to value 1e-09
Default Optimality Tolerance:  1e-06
Set parameter OptimalityTol to value 1e-09
Default MIP Gap Limit:  0.0001
Set parameter MIPGap to value 0
Assigned MIP Gap Limit:  0.0001
The current MIPFocus is:  0
Set parameter MIPFocus to value 2
Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (win64)

CPU model: Intel(R) Core(TM) i3-10110U CPU @ 2.10GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 33 rows, 320 columns and 1067 nonzeros
Model fingerprint: 0x5aa6d6e0
Variable types: 0 continuous, 320 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e-03, 5e+00]
  Objective range  [2e-01, 4e-01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [4e+00, 1e+03]
Presolve removed 0 rows and 6 columns
Presolve time: 0.0

# STANDARDs MINIMIZATION

In [29]:
# STANDARDs MINIMIZATION
M=100000

# Create a new Gurobi model
CSProb3 = gp.Model("Stock Cutting with Unique Standard length usage minimization")

#Debugging Mode
CSProb3.setParam('OutputFlag', 1)

# Variables
x_3 = {}
b_3 = {}

for c in patterns_df:
    x_3[c] = CSProb3.addVar(lb=0, vtype=GRB.INTEGER, name=f'x_3[{c}]')

for sl in unique_elements_row:
    b_3[float(sl)] = CSProb3.addVar(vtype=GRB.BINARY, name=f'b_3[{sl}]')

    
    
    
# Constraints

for sl in unique_elements_row:
    CSProb3.addConstr( gp.quicksum(x_3[c] for c in sl_key__value_c_mapping_dict[sl]) >= b_3[float(sl)])
    CSProb3.addConstr( M*b_3[float(sl)] >= gp.quicksum(x_3[c] for c in sl_key__value_c_mapping_dict[sl]))
    
    

    

# Previous OBJECTIVE 1: Minimize Waste : Now treated as constraint
CSProb3.addConstr(gp.quicksum(x_3[c]*last_two_rows[c]["Wasted Length"] for c in patterns_df) <= ceil(CSProb.objVal *1000)/1000,name="Obj1 as Const.")


for l in all_length_of_cuts_allowed:
    CSProb3.addConstr(gp.quicksum(x_3[c] * new_df[c][l] for c in patterns_df) >= requirements[l],name=f"Requirement const. for {l}")


# Previous Objective 2: Minimize Cost : Now treated as constraint
CSProb3.addConstr(gp.quicksum(x_3[c] * DashBoard_Dataset_StandardLengths["Associated Cost"][float(last_two_rows[c]["Standard Max Length"])] for c in patterns_df) <= ceil(CSProb2.objVal *1000)/1000, name="Obj2 as Const.")


# New Objective 3 to Minimize the number of Unique Standard Length Usage
CSProb3.setObjective(gp.quicksum(b_3[float(sl)] for sl in unique_elements_row),sense=GRB.MINIMIZE)






# Set a time limit for optimization (e.g., 60 seconds)
#https://www.gurobi.com/documentation/10.0/refman/python_parameter_examples.html#PythonParameterExamples
currentTIMElimit = CSProb3.Params.timeLimit
print("Default Time Limit: ",currentTIMElimit)
#CSProb3.Params.timeLimit = Timelimit_for_each_final_CSP_problem_minimizing_WCSP


currentIntFeasTolerancelimit = CSProb3.Params.IntFeasTol
print("Default Interger Tolerance: ",currentIntFeasTolerancelimit)
CSProb3.Params.IntFeasTol = 1e-9


#FeasibilityTol
#https://www.gurobi.com/documentation/current/refman/feasibilitytol.html#parameter:FeasibilityTol
currentFeasibilityTolerancelimit = CSProb3.Params.FeasibilityTol
print("Default Feasibility Tolerance: ",currentFeasibilityTolerancelimit)
CSProb3.Params.FeasibilityTol = 1e-9


#OptimalityTol
#https://www.gurobi.com/documentation/current/refman/optimalitytol.html
currentOptimalityTolerancelimit = CSProb3.Params.OptimalityTol
print("Default Optimality Tolerance: ",currentOptimalityTolerancelimit)
CSProb3.Params.OptimalityTol = 1e-9


currentMIPGapLimit = CSProb3.Params.MIPGap
print("Default MIP Gap Limit: ",currentMIPGapLimit)
#CSProb.setParam('MIPGap', 0)
CSProb3.Params.MIPGap = 0
print("Assigned MIP Gap Limit: ",currentMIPGapLimit)

#https://www.gurobi.com/documentation/current/refman/mip_models.html#:~:text=The%20MIPFocus%20parameter%20allows%20you%20to%20modify%20your,good%20quality%20feasible%20solutions%2C%20you%20can%20select%20MIPFocus%3D1.
#https://www.gurobi.com/documentation/current/refman/mipfocus.html
#https://support.gurobi.com/hc/en-us/community/posts/8791754564369-How-does-MIPFocus-really-work-
currentMIPFocus = CSProb3.Params.MIPFocus
print("The current MIPFocus is: ",currentMIPFocus)
CSProb3.Params.MIPFocus=2

#CSProb3.Params.NumericFocus = 3  # Experiment with different values (0, 1, 2, 3)


# Optimize the model
CSProb3.optimize()

Default Time Limit:  inf
Default Interger Tolerance:  1e-05
Set parameter IntFeasTol to value 1e-09
Default Feasibility Tolerance:  1e-06
Set parameter FeasibilityTol to value 1e-09
Default Optimality Tolerance:  1e-06
Set parameter OptimalityTol to value 1e-09
Default MIP Gap Limit:  0.0001
Set parameter MIPGap to value 0
Assigned MIP Gap Limit:  0.0001
The current MIPFocus is:  0
Set parameter MIPFocus to value 2
Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (win64)

CPU model: Intel(R) Core(TM) i3-10110U CPU @ 2.10GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 42 rows, 324 columns and 2035 nonzeros
Model fingerprint: 0x17e40d67
Variable types: 0 continuous, 324 integer (4 binary)
Coefficient statistics:
  Matrix range     [1e-03, 1e+05]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [4e+00, 1e+03]
Presolve added 4 rows and 3 columns
Presolve time: 0.02s

In [30]:
# IMPORTANT

# #Identify Infeasible Constraints:
# #Use the infeasibility information (model.IIS) to identify the constraints causing infeasibility. This can be done as follows:

# CSProb3.computeIIS()
# print('IIS constraints:')
# for constr in CSProb3.getConstrs():
#     if constr.IISConstr:
#         print(constr.ConstrName)

In [31]:
# Print variable values
for var in CSProb3.getVars():
    print(f'{var.varName}: {var.x}')

# # Print constraint values
# for constr in CSProb3.getConstrs():
#     print(f'{constr.ConstrName}: {constr.Pi} (dual value)')

# Print objective value
print(f'Objective Value: {CSProb3.objVal}')

x_3[Combination 1]: 1.0
x_3[Combination 2]: -0.0
x_3[Combination 3]: -0.0
x_3[Combination 4]: -0.0
x_3[Combination 5]: -0.0
x_3[Combination 6]: -0.0
x_3[Combination 7]: -0.0
x_3[Combination 8]: -0.0
x_3[Combination 9]: -0.0
x_3[Combination 10]: -0.0
x_3[Combination 11]: -0.0
x_3[Combination 12]: -0.0
x_3[Combination 13]: -0.0
x_3[Combination 14]: -0.0
x_3[Combination 15]: -0.0
x_3[Combination 16]: -0.0
x_3[Combination 17]: -0.0
x_3[Combination 18]: -0.0
x_3[Combination 19]: -0.0
x_3[Combination 20]: -0.0
x_3[Combination 21]: -0.0
x_3[Combination 22]: -0.0
x_3[Combination 23]: -0.0
x_3[Combination 24]: -0.0
x_3[Combination 25]: -0.0
x_3[Combination 26]: -0.0
x_3[Combination 27]: -0.0
x_3[Combination 28]: -0.0
x_3[Combination 29]: -0.0
x_3[Combination 30]: -0.0
x_3[Combination 31]: -0.0
x_3[Combination 32]: -0.0
x_3[Combination 33]: -0.0
x_3[Combination 34]: -0.0
x_3[Combination 35]: -0.0
x_3[Combination 36]: -0.0
x_3[Combination 37]: -0.0
x_3[Combination 38]: -0.0
x_3[Combination 39]: -

# PATTERNs MINIMIZATION

In [32]:
# Create a new Gurobi model
CSProb4 = gp.Model("Stock Cutting with Minimum Unique Pattern usage")

# Variables
x_4 = {}
y_4 = {}
b_4 = {}

for c in patterns_df:
    x_4[c] = CSProb4.addVar(lb=0, vtype=GRB.INTEGER, name=f'x_4[{c}]')
    y_4[c] = CSProb4.addVar(vtype=GRB.BINARY, name=f'y_4[{c}]')

for sl in unique_elements_row:
    b_4[sl] = CSProb4.addVar(vtype=GRB.BINARY, name=f'b_4[{sl}]')

# Constraints
for c in patterns_df:
    #CSProb4.addConstr(x_4[c] >= b_4[last_two_rows[c]["Standard Max Length"]]) #Wrong
    CSProb4.addConstr(x_4[c] >= y_4[c])
    #CSProb4.addConstr(M * b_4[last_two_rows[c]["Standard Max Length"]] >= x_4[c]) #Wrong
    CSProb4.addConstr(M * y_4[c] >= x_4[c])

    
# Constraints
for sl in unique_elements_row:
    CSProb4.addConstr( gp.quicksum(x_4[c] for c in sl_key__value_c_mapping_dict[sl]) >= b_4[float(sl)])
    CSProb4.addConstr( M*b_4[float(sl)] >= gp.quicksum(x_4[c] for c in sl_key__value_c_mapping_dict[sl]))
    
    
# Previous OBJECTIVE 1: Minimize Waste : Now treated as constraint
CSProb4.addConstr(gp.quicksum(x_4[c]*last_two_rows[c]["Wasted Length"] for c in patterns_df) <= ceil(CSProb.objVal *1000)/1000)

# Satisfying Requirements
for l in all_length_of_cuts_allowed:
    CSProb4.addConstr(gp.quicksum(x_4[c] * new_df[c][l] for c in patterns_df) >= requirements[l])

# Previous Objective 2: Minimize Cost : Now treated as constraint
CSProb4.addConstr(gp.quicksum(x_4[c] * DashBoard_Dataset_StandardLengths["Associated Cost"][float(last_two_rows[c]["Standard Max Length"])] for c in patterns_df) <= ceil(CSProb2.objVal *1000)/1000)




# Previous Objective 3: to Minimize the number of Unique Standard Length Usage
CSProb4.addConstr(gp.quicksum(b_4[sl] for sl in unique_elements_row) <= ceil(CSProb3.objVal *1000)/1000)


# New Objective 4 to Minimize the number of Unique Pattern Usage
CSProb4.setObjective(gp.quicksum(y_4[c] for c in patterns_df),sense=GRB.MINIMIZE)





# Set a time limit for optimization (e.g., 60 seconds)
#https://www.gurobi.com/documentation/10.0/refman/python_parameter_examples.html#PythonParameterExamples
currentTIMElimit = CSProb4.Params.timeLimit
print("Default Time Limit: ",currentTIMElimit)
#CSProb4.Params.timeLimit = Timelimit_for_each_final_CSP_problem_minimizing_WCSP


currentIntFeasTolerancelimit = CSProb4.Params.IntFeasTol
print("Default Interger Tolerance: ",currentIntFeasTolerancelimit)
CSProb4.Params.IntFeasTol = 1e-9


#FeasibilityTol
#https://www.gurobi.com/documentation/current/refman/feasibilitytol.html#parameter:FeasibilityTol
currentFeasibilityTolerancelimit = CSProb4.Params.FeasibilityTol
print("Default Feasibility Tolerance: ",currentFeasibilityTolerancelimit)
CSProb4.Params.FeasibilityTol = 1e-9


#OptimalityTol
#https://www.gurobi.com/documentation/current/refman/optimalitytol.html
currentOptimalityTolerancelimit = CSProb4.Params.OptimalityTol
print("Default Optimality Tolerance: ",currentOptimalityTolerancelimit)
CSProb4.Params.OptimalityTol = 1e-9


currentMIPGapLimit = CSProb4.Params.MIPGap
print("Default MIP Gap Limit: ",currentMIPGapLimit)
#CSProb.setParam('MIPGap', 0)
CSProb4.Params.MIPGap = 0
print("Default MIP Gap Limit: ",currentMIPGapLimit)

#https://www.gurobi.com/documentation/current/refman/mip_models.html#:~:text=The%20MIPFocus%20parameter%20allows%20you%20to%20modify%20your,good%20quality%20feasible%20solutions%2C%20you%20can%20select%20MIPFocus%3D1.
#https://www.gurobi.com/documentation/current/refman/mipfocus.html
#https://support.gurobi.com/hc/en-us/community/posts/8791754564369-How-does-MIPFocus-really-work-
currentMIPFocus = CSProb4.Params.MIPFocus
print("The current MIPFocus is: ",currentMIPFocus)
CSProb4.Params.MIPFocus=2


# Optimize the model
CSProb4.optimize()

Default Time Limit:  inf
Default Interger Tolerance:  1e-05
Set parameter IntFeasTol to value 1e-09
Default Feasibility Tolerance:  1e-06
Set parameter FeasibilityTol to value 1e-09
Default Optimality Tolerance:  1e-06
Set parameter OptimalityTol to value 1e-09
Default MIP Gap Limit:  0.0001
Set parameter MIPGap to value 0
Default MIP Gap Limit:  0.0001
The current MIPFocus is:  0
Set parameter MIPFocus to value 2
Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (win64)

CPU model: Intel(R) Core(TM) i3-10110U CPU @ 2.10GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 683 rows, 644 columns and 3319 nonzeros
Model fingerprint: 0x112fd8bd
Variable types: 0 continuous, 644 integer (324 binary)
Coefficient statistics:
  Matrix range     [1e-03, 1e+05]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e+00, 1e+03]
Presolve added 4 rows and 4 columns
Presolve time: 0.0

# Populate Result Excel Sheet

In [33]:
end_time_is_this=time.time()
print(end_time_is_this)

actual_time_taken = end_time_is_this - start_time_is_this
actual_time_taken

1702800407.8312697


326.3915150165558

In [34]:
winsound.Beep(500,1000)
wb_final=openpyxl.Workbook()

sheet_for_Usage = wb_final.active
sheet_for_Usage.title = 'Variable x and y'
sheet_for_b = wb_final.create_sheet(title='Variable b')
sheet_for_Objectives = wb_final.create_sheet(title='Objectives')

#sheet_for_x
row_number=1
sheet_for_Usage.cell(row=row_number,column=1).value= "PATTERN"
sheet_for_Usage.cell(row=row_number,column=2).value= "Standard Length"
sheet_for_Usage.cell(row=row_number,column=3).value= "No. of Usage in Objective 1 (x)"
sheet_for_Usage.cell(row=row_number,column=4).value= "No. of Usage in Objective 2 (x_2)"
sheet_for_Usage.cell(row=row_number,column=5).value= "No. of Usage in Objective 3 (x_3)"
sheet_for_Usage.cell(row=row_number,column=6).value= "No. of Usage in Objective 4 (x_4)"
sheet_for_Usage.cell(row=row_number,column=7).value= "Unique Pattern Usages in Objective 4 (y_4)"

for c in patterns_df:
    row_number+=1
    sheet_for_Usage.cell(row=row_number,column=1).value=c
    sheet_for_Usage.cell(row=row_number,column=2).value=last_two_rows[c]["Standard Max Length"]
    sheet_for_Usage.cell(row=row_number,column=3).value=x[c].x
    sheet_for_Usage.cell(row=row_number,column=4).value=x_2[c].x
    sheet_for_Usage.cell(row=row_number,column=5).value=x_3[c].x
    sheet_for_Usage.cell(row=row_number,column=6).value=x_4[c].x
    sheet_for_Usage.cell(row=row_number,column=7).value=y_4[c].x
    #if x_3[c].x:
    #    print(c, " used " , x_3[c].x , "times, with standard length", last_two_rows[c]["Standard Max Length"])


    
#sheet_for_b
row_number=1
sheet_for_b.cell(row=row_number,column=1).value= "Standard Length"
sheet_for_b.cell(row=row_number,column=2).value= "Utilization in Objective 3 (b-3)"
sheet_for_b.cell(row=row_number,column=3).value= "Utilization in Objective 4 (b_4)"

for sl in unique_elements_row:
    row_number+=1
    sheet_for_b.cell(row=row_number,column=1).value=sl
    sheet_for_b.cell(row=row_number,column=2).value=b_3[sl].x
    sheet_for_b.cell(row=row_number,column=3).value=b_4[sl].x
    
    
#sheet_for_Objectives
row_number=1
sheet_for_Objectives.cell(row=row_number,column=1).value= "Objective 1"
sheet_for_Objectives.cell(row=row_number,column=2).value= "Objective 2"
sheet_for_Objectives.cell(row=row_number,column=3).value= "Objective 3"
sheet_for_Objectives.cell(row=row_number,column=4).value= "Objective 4"
sheet_for_Objectives.cell(row=row_number,column=5).value= "Total Time taken for the Pattern Generations and all Objective Computations"
row_number=2
sheet_for_Objectives.cell(row=row_number,column=1).value= "Minimum Total WASTE"
sheet_for_Objectives.cell(row=row_number,column=2).value= "Minimum Total COST"
sheet_for_Objectives.cell(row=row_number,column=3).value= "Minimum Unique STANDARD LENGTH Usage"
sheet_for_Objectives.cell(row=row_number,column=4).value= "Minimum Unique PATTERN Usage"
sheet_for_Objectives.cell(row=row_number,column=5).value= "(Seconds)"
row_number=3
sheet_for_Objectives.cell(row=row_number,column=1).value= CSProb.objVal
sheet_for_Objectives.cell(row=row_number,column=2).value= CSProb2.objVal
sheet_for_Objectives.cell(row=row_number,column=3).value= CSProb3.objVal
sheet_for_Objectives.cell(row=row_number,column=4).value= CSProb4.objVal
sheet_for_Objectives.cell(row=row_number,column=5).value= actual_time_taken


Final_output_file_path = "./Prob"+str(Decisions["Problem"])+"_PatternUsage_with_WCSP_Minimization"+".xlsx"
wb_final.save(Final_output_file_path)

In [35]:
# winsound.Beep(500,1000)
# wb_final=openpyxl.Workbook()

# sheet_for_Usage = wb_final.active

# row_number=1
# sheet_for_Usage.cell(row=row_number,column=1).value= "PATTERN"
# sheet_for_Usage.cell(row=row_number,column=2).value= "Standard Length"
# sheet_for_Usage.cell(row=row_number,column=3).value= "No. of Usage"


# for c in patterns_df:
#     row_number+=1
#     sheet_for_Usage.cell(row=row_number,column=1).value=c
#     sheet_for_Usage.cell(row=row_number,column=2).value=last_two_rows[c]["Standard Max Length"]
#     sheet_for_Usage.cell(row=row_number,column=3).value=x[c].x
#     if x[c].x:
#         print(c, " used " , x[c].x , "times, with standard length", last_two_rows[c]["Standard Max Length"])

        
# row_number+=3
# sheet_for_Usage.cell(row=row_number,column=1).value= "Minimum Trim Loss"
# sheet_for_Usage.cell(row=row_number,column=2).value= "Objective Value:"
# sheet_for_Usage.cell(row=row_number,column=3).value= CSProb.objVal
# print("Objective Value:", CSProb.objVal)

# Final_output_file_path = "./Problem_"+str(Decisions["Problem"])+"__PatternUsage_withMinimumTrimLoss"+"_Gurobi.xlsx"
# wb_final.save(Final_output_file_path)

In [36]:
# winsound.Beep(500,1000)
# wb_final=openpyxl.Workbook()

# sheet_for_Usage = wb_final.active

# row_number=1
# sheet_for_Usage.cell(row=row_number,column=1).value= "PATTERN"
# sheet_for_Usage.cell(row=row_number,column=2).value= "Standard Length"
# sheet_for_Usage.cell(row=row_number,column=3).value= "No. of Usage"


# for c in patterns_df:
#     row_number+=1
#     sheet_for_Usage.cell(row=row_number,column=1).value=c
#     sheet_for_Usage.cell(row=row_number,column=2).value=last_two_rows[c]["Standard Max Length"]
#     sheet_for_Usage.cell(row=row_number,column=3).value=x_2[c].x
#     if x_2[c].x:
#         print(c, " used " , x_2[c].x , "times, with standard length", last_two_rows[c]["Standard Max Length"])
        
# row_number+=3
# sheet_for_Usage.cell(row=row_number,column=1).value= "Minimum Total Cost"
# sheet_for_Usage.cell(row=row_number,column=2).value= "Objective Value:"
# sheet_for_Usage.cell(row=row_number,column=3).value= CSProb2.objVal
# print("Objective Value:", CSProb2.objVal)

# Final_output_file_path = "./Problem_"+str(Decisions["Problem"])+"__PatternUsage_withMinimumCost"+"_Gurobi.xlsx"
# wb_final.save(Final_output_file_path)